In [1]:
import numpy as np
import tensorflow as tf
import librosa
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,Add
from keras.utils import np_utils
from IPython.display import Audio
from scipy.io import wavfile
import pandas as pd
import numpy as np
from IPython.display import Audio
from scipy.io import wavfile
import IPython.display as ipd

Using TensorFlow backend.


Cloning the data repository from github

In [2]:
!git clone https://github.com/sanath493/dl-as1-audio.git

Cloning into 'dl-as1-audio'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


Extracting files from the zip file

In [0]:
from zipfile import ZipFile
file_name = "dl-as1-audio/data.zip"
with ZipFile(file_name , 'r') as zip:
  zip.extractall()

Converting the Audio signals into spectograms

In [0]:
s, sr=librosa.load('train_clean_male.wav', sr=None)
S=librosa.stft(s, n_fft=1024, hop_length=512)
S=np.transpose(S)
S=np.abs(S)
 
sn, sr=librosa.load('train_dirty_male.wav', sr=None)
X=librosa.stft(sn, n_fft=1024, hop_length=512)
X=np.transpose(X)
X=np.abs(X)

sl, sr=librosa.load('test_x_01.wav', sr=None)
Y=librosa.stft(sl, n_fft=1024, hop_length=512)
Y=np.transpose(Y)
Y=np.abs(Y)


sa, sr=librosa.load('test_x_02.wav', sr=None)
Z=librosa.stft(sa, n_fft=1024, hop_length=512)
Z=np.transpose(Z)
Z=np.abs(Z)


In [5]:
S.shape[1]

513

In [0]:
print(X.shape)
S.shape

(2459, 513)


(2459, 513)

Defining a model and training it over the stft matrices 0f given dirty and clean audio signals

In [0]:
#X = tf.keras.utils.normalize(X)
#X_test = tf.keras.utils.normalize(X_test)

model =Sequential()
model.add(Dense(1024,activation=tf.nn.relu,input_shape=(513,),kernel_initializer='glorot_normal'))
model.add(Dense(1024,activation=tf.nn.relu))
model.add(Dense(513,activation=tf.nn.relu))
model.compile(optimizer='adam',loss='mse', metrics=['accuracy'])
model.summary()
model.fit(X,S,epochs = 100,batch_size=100)






Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              526336    
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dense_3 (Dense)              (None, 513)               525825    
Total params: 2,101,761
Trainable params: 2,101,761
Non-trainable params: 0
_________________________________________________________________



Epoch 1/100





2459/2459 [==============================] - 5s 2ms/step - loss: 0.0426 - acc: 0.1651
Epoch 2/100
2459/2459 [==============================] - 0s 81us/step - loss: 0.0179 - acc: 0.3355
Epoch 3/100
2459/2459 [==============================] - 0s 78us/step - loss: 0.0123 - acc: 0.3953
Epoch 4/100
2459/2459 [==============================] - 0s 78us/

Predicting the output of a given test audio(test_x_01.wav) using the trained model

And applying inverse stft to recover the audio signal

In [0]:
y_pred1 = model.predict(Y)
K=np.transpose(y_pred1)
sp, sr=librosa.load('test_x_01.wav', sr=None)
Z1=librosa.stft(sp, n_fft=1024, hop_length=512)
Z2=np.abs(Z1)
z=(Z1/Z2)
z.shape
o=np.multiply(z,K)
Op=librosa.istft(o,hop_length=512)

Creating an audio file from the predicted output from model

In [0]:
librosa.output.write_wav('test_s_01_recons1.wav', Op, sr)

1st test input for reference

In [0]:
ipd.Audio('test_x_01.wav')

Cleaned audio ouput from the model

In [0]:
ipd.Audio('test_s_01_recons1.wav')

Predicting the output of a given test audio(test_x_02.wav) using the trained model

And applying inverse stft to recover the audio signal

In [0]:
y_pred2 = model.predict(Z)
K1=np.transpose(y_pred2)
sa, sr=librosa.load('test_x_02.wav', sr=None)
Y1=librosa.stft(sa, n_fft=1024, hop_length=512)
Y2=np.abs(Y1)
C=(Y1/Y2)
C.shape
L=np.multiply(C,K1)
Op1=librosa.istft(L,hop_length=512)

Creating an audio file from the predicted output from model

In [0]:
librosa.output.write_wav('test_s_01_recons2.wav', Op1, sr)

2nd test input for reference

In [0]:
ipd.Audio('test_x_02.wav')

Cleaned audio ouput from the model

In [0]:
ipd.Audio('test_s_01_recons2.wav')